In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [3]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [4]:
import gzip
import simplejson as json

In [5]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [6]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

In [7]:
[script for script in scripts if script['bnf_name'] == 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F']

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101010G0AAABAB',
  'items': 1,
  'practice': 'N81029',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 2.99,
  'act_cost': 2.78,
  'quantity': 500},
 {'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81088',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101010G0AAABAB',
  'items': 6,
  'practice': 'A81017',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 20.93,
  'act_cost': 19.45,
  'quantity': 3500},
 {'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'A81034',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101010G0AAABAB',
  'items': 10,
  'practice': 'P85003',
  'bnf_name': 'Co

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [8]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [9]:
import statistics

def describe(key):
    
    values = [prescription[key] for prescription in scripts]
    middle = len(values) // 2
    # print(middle)
    # print(len(values))
    # print(values[:20])
    # print(sorted(values)[:39])
    
    total = sum(values)
    avg = total / len(values)
    s = statistics.stdev(values)
    q25 = statistics.median((sorted(values))[:middle])
    med = statistics.median(values)
    q75 = statistics.median((sorted(values))[middle:]) if len(values)%2==0 else statistics.median((sorted(values))[middle+1:])

    return [total, avg, s, q25, med, q75]

In [10]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]
summary

[('items', [4410054, 11.522744731217633, 33.11220959819492, 1, 3.0, 8]),
 ('quantity',
  [316356836, 826.5883059943667, 3872.1860733044696, 30, 120.0, 466]),
 ('nic',
  [29048309.790000338,
   75.89844899484315,
   197.57308474088757,
   7.7,
   22.62,
   65.94]),
 ('act_cost',
  [27053937.599999707,
   70.68748295124895,
   183.26755837715774,
   7.25,
   21.24,
   61.53])]

In [11]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [12]:
list_bnf_names = [script['bnf_name'] for script in scripts]
# print(list_bnf_names)
# print(len(list_bnf_names))
# print(list_bnf_names)

set_bnf_names = set(list_bnf_names)
# print(len(set_bnf_names))
# print(set_bnf_names)

In [13]:
bnf_names = set([script['bnf_name'] for script in scripts])
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [14]:
groups = {name: [] for name in bnf_names}

for script in scripts:
    groups[script['bnf_name']].append(script)


Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [15]:
dict_bnf_items = {k: sum([script['items'] for script in v]) for k, v in groups.items()}

In [16]:
import operator
max_item = [max(dict_bnf_items.items(), key=operator.itemgetter(1))]

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [17]:
grader.score.pw__most_common_item(max_item)

Error!
You have been rate limited for exceeding the limit of 1 per 1 second.
Please slow down your submission rate.


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [18]:
def group_by_field(data, fields):
    groups = {name: [] for name in set([item[fields[0]] for item in data])}
    
    for obj in data:
        groups[obj[fields[0]]].append(obj)
    
    return groups


In [19]:
import operator

groups = group_by_field(scripts, ('bnf_name',))
dict_bnf_items = {k: sum([script['items'] for script in v]) for k, v in groups.items()}

test_max_item =  [max(dict_bnf_items.items(), key=operator.itemgetter(1))]

assert test_max_item == max_item

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [20]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        if (practice_postal[practice['code']]) > (practice['post_code']):
            practice_postal[practice['code']] = practice['post_code']
    else:
        practice_postal[practice['code']] = practice['post_code']

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [21]:
assert practice_postal['K82019'] == 'HP21 8TR'

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [22]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [23]:
joined = scripts[:]
for script in joined:
    script['post_code'] = practice_postal[script['practice']]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [24]:
items_by_post = group_by_field(joined, ('post_code', )) # NOTE: items_by_post is more accurately, prescriptions grouped by post_code

# li = []
# items_by_post['BB7 2JG']
# for obj in items_by_post['BB7 2JG']:
#     if obj['bnf_code'] == '0101021B0AAAPAP':
#         li.append(obj)

# print(len(li))
# li

In [25]:
dict_postcode_items = {k: sum([pescription['items'] for pescription in v]) for k, v in items_by_post.items()}

# converting to a list of tuples
list_of_tuples_of_postcode_items = list(dict_postcode_items.items())

# sorting the list of tuples
list_of_tuples_of_postcode_items.sort(key = lambda e: e[0])


print(list_of_tuples_of_postcode_items)

[('B11 4BW', 20673), ('B18 7AL', 19001), ('B21 9RY', 29103), ('B23 6DJ', 24859), ('B70 7AW', 36531), ('BB11 2DL', 34356), ('BB2 1AX', 28254), ('BB3 1PY', 54514), ('BB4 5SL', 29388), ('BB7 2JG', 44585), ('BB8 0JZ', 54380), ('BB9 7SR', 38224), ('BD3 8QH', 21010), ('BH18 8EE', 39413), ('BH23 3AF', 32545), ('BL1 8TU', 26132), ('BL3 5HP', 27147), ('BL9 0NJ', 32062), ('BL9 0SN', 35275), ('CB9 8HF', 51337), ('CH1 4DS', 34915), ('CH65 6TG', 25090), ('CT11 8AD', 44358), ('CV1 4FS', 37210), ('CW1 3AW', 64104), ('CW5 5NX', 38797), ('CW7 1AT', 43164), ('DA1 2HA', 26075), ('DA11 8BZ', 24090), ('DN22 7XF', 43091), ('DN34 4GB', 48013), ('FY2 0JG', 69118), ('FY4 1TJ', 62886), ('FY5 2TZ', 44258), ('FY7 8GU', 34473), ('GL1 3PX', 38120), ('GL50 4DP', 74822), ('GU9 9QS', 32131), ('HA0 4UZ', 22755), ('HA3 7LT', 32113), ('HG1 5AR', 32684), ('HU7 4DW', 49107), ('KT14 6DH', 26758), ('KT6 6EZ', 38975), ('L31 0DJ', 32065), ('L36 7XY', 22965), ('L5 0QW', 24676), ('L7 6HD', 42569), ('LA1 1PN', 47335), ('LE10 1DS'

In [26]:
postal_totals = list_of_tuples_of_postcode_items[:100]

grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [27]:
post_bnf_items = []
for key in items_by_post:
    item_by_bnfname_per_post = group_by_field(items_by_post[key], ('bnf_name',))

    items_prescribed_per_bnf_post = [(key, k, sum([pescription['items'] for pescription in v])) for k, v in item_by_bnfname_per_post.items()]
    post_bnf_items.extend(items_prescribed_per_bnf_post)
    

len(post_bnf_items)
post_bnf_items[-10:]


[('WN2 5NG', 'Pepti_Jnr Pdr', 4),
 ('WN2 5NG', 'Desloratadine_Tab 5mg', 2),
 ('WN2 5NG', 'Nystan_Susp (Ready Mixed)', 7),
 ('WN2 5NG', 'Dermol_Crm', 16),
 ('WN2 5NG', 'Ramipril_Tab 10mg', 25),
 ('WN2 5NG', 'Co-Careldopa_Tab 25mg/100mg', 7),
 ('WN2 5NG', 'Co-Trimoxazole_Tab 80mg/400mg', 3),
 ('WN2 5NG', 'Coloplast SpeediCath Male Size 10-14 (30', 7),
 ('WN2 5NG', 'Candesartan Cilexetil_Tab 8mg', 28),
 ('WN2 5NG', 'Pramipexole_Tab 88mcg', 8)]

In [28]:
total_items_by_bnf_post = post_bnf_items
assert len(total_items_by_bnf_post) == 141196

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [29]:
dict_items_by_bnf_post = {k: [] for k in dict_postcode_items}

keys_ = ('bnf_name', 'items')
for ele in total_items_by_bnf_post:
    dict_items_by_bnf_post[ele[0]].append(dict(list(zip(keys_, (ele[1], ele[2])))))


total_items_by_post = {key: sum([obj['items'] for obj in dict_items_by_bnf_post[key]]) for key in dict_items_by_bnf_post}
total_items_by_post = dict(sorted(total_items_by_post.items()))
total_items_by_post

{'B11 4BW': 20673,
 'B18 7AL': 19001,
 'B21 9RY': 29103,
 'B23 6DJ': 24859,
 'B70 7AW': 36531,
 'BB11 2DL': 34356,
 'BB2 1AX': 28254,
 'BB3 1PY': 54514,
 'BB4 5SL': 29388,
 'BB7 2JG': 44585,
 'BB8 0JZ': 54380,
 'BB9 7SR': 38224,
 'BD3 8QH': 21010,
 'BH18 8EE': 39413,
 'BH23 3AF': 32545,
 'BL1 8TU': 26132,
 'BL3 5HP': 27147,
 'BL9 0NJ': 32062,
 'BL9 0SN': 35275,
 'CB9 8HF': 51337,
 'CH1 4DS': 34915,
 'CH65 6TG': 25090,
 'CT11 8AD': 44358,
 'CV1 4FS': 37210,
 'CW1 3AW': 64104,
 'CW5 5NX': 38797,
 'CW7 1AT': 43164,
 'DA1 2HA': 26075,
 'DA11 8BZ': 24090,
 'DN22 7XF': 43091,
 'DN34 4GB': 48013,
 'FY2 0JG': 69118,
 'FY4 1TJ': 62886,
 'FY5 2TZ': 44258,
 'FY7 8GU': 34473,
 'GL1 3PX': 38120,
 'GL50 4DP': 74822,
 'GU9 9QS': 32131,
 'HA0 4UZ': 22755,
 'HA3 7LT': 32113,
 'HG1 5AR': 32684,
 'HU7 4DW': 49107,
 'KT14 6DH': 26758,
 'KT6 6EZ': 38975,
 'L31 0DJ': 32065,
 'L36 7XY': 22965,
 'L5 0QW': 24676,
 'L7 6HD': 42569,
 'LA1 1PN': 47335,
 'LE10 1DS': 49335,
 'LE18 2EW': 37144,
 'LE5 3GH': 28654,
 '

In [30]:
total_items = total_items_by_post
assert len(total_items_by_post) == 118

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [40]:

def get_max_item_in_region(li):
    # sort by bnf_name in alphabetical-descending order: Z, Y, X, ...C, B, A
    li_1 = sorted(li, key=operator.itemgetter(1), reverse=True)
    
    # sort in ascending order of items number: 0.0, 0.1, ....
    li_2 = sorted(li_1, key=operator.itemgetter(2))
    return li_2.pop()

In [49]:
# create a new list of tuples similar to post_bnf_items but with the items expressed as a proportion of
# of the regional total. This is achieved by dividing the item value by the corresponding regional total gotten

post_bnf_proptn_item = [(ele[0], ele[1], ele[2]/total_items_by_post[ele[0]]) for ele in post_bnf_items]

In [56]:
# filter the post_bnf_proptn_item by region (ie. by post_code which is the element index 0 in post_bnf_proptn_item)
# hence the x[0] in the filter() below; then for each region, we get the tuple with the maximum proptional item using
# the get_max_item_in_region() function; next we add the regional maximum item (as a tuple of the post_code, bnf_name,
# and proportnal_value) to the results list

results = []
for key in total_items_by_post:
    regional_values = list(filter(lambda x: x[0] == key, post_bnf_proptn_item))
    results.append(get_max_item_in_region(regional_values))

# print(len(results))
results

[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.03415082474725487),
 ('B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.02926161780958897),
 ('B21 9RY', 'Metformin HCl_Tab 500mg', 0.03549462254750369),
 ('B23 6DJ', 'Lansoprazole_Cap 30mg (E/C Gran)', 0.024095900880968663),
 ('B70 7AW', 'Paracet_Tab 500mg', 0.0266896608360023),
 ('BB11 2DL', 'Omeprazole_Cap E/C 20mg', 0.02884503434625684),
 ('BB2 1AX', 'Omeprazole_Cap E/C 20mg', 0.03645501521908402),
 ('BB3 1PY', 'Omeprazole_Cap E/C 20mg', 0.03428477088454342),
 ('BB4 5SL', 'Omeprazole_Cap E/C 20mg', 0.040696883081529876),
 ('BB7 2JG', 'Omeprazole_Cap E/C 20mg', 0.029471795446899183),
 ('BB8 0JZ', 'Atorvastatin_Tab 20mg', 0.022563442442074293),
 ('BB9 7SR', 'Omeprazole_Cap E/C 20mg', 0.023833193804939305),
 ('BD3 8QH', 'Atorvastatin_Tab 40mg', 0.03422179914326511),
 ('BH18 8EE', 'Omeprazole_Cap E/C 20mg', 0.029000583563798747),
 ('BH23 3AF', 'Omeprazole_Cap E/C 20mg', 0.03733292364418497),
 ('BL1 8TU', 'Omeprazole_Cap E/C 20mg', 0

In [53]:
max_item_by_post = results

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [54]:
# take the first hundred results
items_by_region = max_item_by_post[:100]

In [55]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*